In [ ]:
!pip install python-dotenv

In [1]:
pip install --upgrade openai

In [33]:
import openai as OpenAI
import csv
import os
from dotenv import load_dotenv
import requests
import json
import random

env_path = "/content/drive/MyDrive/Vezilka/gpt_3.5_turbo/.env"
load_dotenv(env_path)

OPENROUTER_API_KEY = os.getenv("OPENROUTER_API_KEY")

if not OPENROUTER_API_KEY:
    raise ValueError("OPENROUTER_API_KEY environment variable not set")

PROMPT = (
    "Generate 10 short Macedonian social media posts labeled as 1 (hate speech) **only if the sentence attacks or insults someone or a group**\n"
    "not if it talks about hate in general or 0 (not hate speech). "
    "No numbering, bullets, or quotes. Format as: post text,1 or post text,0\n"
    "Example:\n"
    "Денес беше убав ден со пријателите, уживавме во сонцето.,0\n"
    "Тој/таа е ужасен и никогаш не треба да биде дел од оваа заедница.,1\n"
)

NUM_OF_CALLS = 60


def generate_text():
    url = "https://openrouter.ai/api/v1/chat/completions"

    headers = {
        "Authorization": f"Bearer {OPENROUTER_API_KEY}",
        "HTTP-Referer": "http://localhost",
        "X-Title": "movie-review-generator",
        "Content-Type": "application/json",
    }

    payload = {
        "model": "openai/gpt-3.5-turbo",
        "messages": [
            {"role": "system", "content": "You are a strict CSV data generator."},
            {"role": "user", "content": PROMPT},
        ],
        "temperature": 0.7,
    }

    response = requests.post(url, headers=headers, json=payload)
    response.raise_for_status()

    return response.json()["choices"][0]["message"]["content"].strip()

def save_to_csv(data, file_name):
    with open(file_name, mode="w", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        writer.writerow(["text", "label"])
        for row in data:
            writer.writerow(row)

if __name__ == "__main__":
    hate = []
    non_hate = []

    try:
        for i in range(NUM_OF_CALLS):
            response = generate_text()

            for line in response.split("\n"):
                if not line.strip():
                    continue
                try:
                    text, label = line.rsplit(",", 1)
                    label = int(label.strip())

                    if label == 1:
                        hate.append((text.strip(), 1))
                    elif label == 0:
                        non_hate.append((text.strip(), 0))
                except ValueError:
                    print(f"Skipping invalid line: {line}")

            print(f"Completed API call {i + 1} of {NUM_OF_CALLS}")

    except KeyboardInterrupt:
        print("\nKeyboard interrupt detected. Saving partial data...")

    finally:
        min_size = min(len(hate), len(non_hate))
        balanced_data = hate[:min_size] + non_hate[:min_size]

        # ─────────── DETECT DUPLICATES ───────────
        from sklearn.feature_extraction.text import TfidfVectorizer
        import numpy as np

        texts = [text for text, _ in balanced_data]
        vectorizer = TfidfVectorizer()
        X = vectorizer.fit_transform(texts)
        similarity_matrix = (X * X.T).toarray()

        duplicates = np.where(np.triu(similarity_matrix, k=1) > 0.9)
        duplicate_indices = set()
        for i, j in zip(*duplicates):
            duplicate_indices.add(j)

        print(f"Found {len(duplicate_indices)} duplicates out of {len(balanced_data)} sentences")
        balanced_data = [s for idx, s in enumerate(balanced_data) if idx not in duplicate_indices]

        random.shuffle(balanced_data)

        output_path = "/content/drive/MyDrive/Vezilka/gpt_3.5_turbo/output_hate_speech.csv"
        save_to_csv(balanced_data, output_path)

        print(
            f"Data saved to '{output_path}' "
            f"(balanced: {min_size} hate / {min_size} non-hate)"
        )

Completed API call 1 of 60
Completed API call 2 of 60
Completed API call 3 of 60
Completed API call 4 of 60
Completed API call 5 of 60
Completed API call 6 of 60
Completed API call 7 of 60
Completed API call 8 of 60
Completed API call 9 of 60
Completed API call 10 of 60
Completed API call 11 of 60
Completed API call 12 of 60
Completed API call 13 of 60
Completed API call 14 of 60
Completed API call 15 of 60
Completed API call 16 of 60
Completed API call 17 of 60
Completed API call 18 of 60
Completed API call 19 of 60
Completed API call 20 of 60
Completed API call 21 of 60
Completed API call 22 of 60
Completed API call 23 of 60
Completed API call 24 of 60
Completed API call 25 of 60
Completed API call 26 of 60
Completed API call 27 of 60
Completed API call 28 of 60
Completed API call 29 of 60
Completed API call 30 of 60
Completed API call 31 of 60
Completed API call 32 of 60
Completed API call 33 of 60
Completed API call 34 of 60
Completed API call 35 of 60
Completed API call 36 of 60
C

In [34]:
df = pd.read_csv("/content/drive/MyDrive/Vezilka/gpt_3.5_turbo/output_hate_speech.csv")

In [35]:
df

,text,label
0,Сите кои го поддржуваат власта се корумпирани.,1
1,Секој што ги има над 60 години е заостанат и н...,1
2,Сите што се борат за правдата се наивни и глуп...,0
3,Сите нивни постови се толку безсмислени и идио...,1
4,Најдобриот начин да го решиме проблемот е да г...,0
...,...,...
254,"Оваа фотографија наистина не ти прилега, изгле...",1
255,"Оваа заедница е толку топла и поддржувачка, бл...",0
256,Не ми се допаѓа како се однесуваат многумина в...,0
257,Сите студенти се лениви и не треба да ги поддр...,1


In [36]:
df['text'][:20].tolist()

['Сите кои го поддржуваат власта се корумпирани.',
 'Секој што ги има над 60 години е заостанат и не разбира од технологија.',
 'Сите што се борат за правдата се наивни и глупави.',
 'Сите нивни постови се толку безсмислени и идиотски, треба да ги избришат.',
 'Најдобриот начин да го решиме проблемот е да ги сите работиме заедно.',
 'Сите кои се разликуваат од нас се непријатели на нашата нација.',
 'Треба да ги изгониме сите кои не се согласуваат со нашите вредности.',
 'Најдобар ден за излез, дружење и уживање!',
 'Сите мигранти треба да го напуштат земјата',
 'Да се бориме за еднакви права за сите групи во заедницата.',
 'Колку пати треба да ви кажам дека сите треба да бидеме заедно?',
 'Овие учители се секогаш непрофесионални и непријатни.',
 'Ти си најглупа особа која ја познавам.',
 'Жените треба да го задржат своето мислење за себе.',
 'Сите мигранти се криминалци кои го угрожуваат нашето општество!',
 'Секој ко не го поддржува нашиот тим е глуп.',
 'Дајте го искоренувањето на с